# Allignment


## Import Modules

##### note `os.environ['DISPLAY']` sets up an empty link with headless server, and depends upon use of XPRA software.  See notes under '...ClearMap2/ClearMap/Headless.txt'

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
os.environ['DISPLAY'] = ':999'
sys.path.append('/home/zachpen87/clearmap/ClearMap2/')

import dask.array as da
import fnmatch
import holoviews as hv
import h5py
from ClearMap.Environment import * 
from holoviews import streams
import ClearMap.IO.TIF_to_HDF5 as H5converter

hv.notebook_extension('bokeh')

## Initialize workspace

In [ ]:
animal = '107'
directory = '/home/zachpen87/clearmap/data/SEFL17b/ms_{id}/'.format(id=animal) 

#directory in which tiff folders are stored
remote_directory = '/media/share/csstorage/Zach/LargeFiles/iDISCO/SEFL_17b/ms_{id}'.format(id=animal)

os.makedirs(directory) if not os.path.isdir(directory) else None
ws = wsp.Workspace('CellMap', directory=directory)
hdf5_file = os.path.join(directory,'data.hdf5')
resources_directory = settings.resources_path
ws.info()

## Convert tif folders to hdf5 datasets

In [ ]:
%%time

buffer_size = 200
if ws.file_list('hdf5')==[]:
    
    H5converter.tiffolder_tohdf5(
        directory = os.path.join(
            remote_directory,
            fnmatch.filter(os.listdir(remote_directory),'*647*')[0]),
        directory_out = directory,
        output_file = 'data.hdf5', 
        dsetname = 'fos_647',
        buffer_size = buffer_size)
    
    H5converter.tiffolder_tohdf5(
        directory = os.path.join(
            remote_directory,
            fnmatch.filter(os.listdir(remote_directory),'*790*')[0]),
        directory_out = directory,
        output_file = 'data.hdf5', 
        dsetname = 'fos_790',
        buffer_size = buffer_size)
    
    H5converter.tiffolder_tohdf5(
        directory = os.path.join(
            remote_directory,
            fnmatch.filter(os.listdir(remote_directory),'*auto*')[0]),
        directory_out = directory,
        output_file = 'data.hdf5', 
        dsetname = 'auto',
        buffer_size = buffer_size)
    ws.update(hdf5 = 'data.hdf5')
    
ws.info()

## Define cropping bounds from max projection

#### Calculate max projection along desired axis

In [ ]:
with h5py.File(hdf5_file,'a') as f:
    maxp = da.from_array(f['auto'],chunks=(500,500,500)).max(axis=0).compute()

#### Display max image.  Crop using box selection tool

In [ ]:
%%output size = 25
cropping_image, cropping_stream = jvis.crop_img(maxp, lims=(0,1000))
cropping_image

#### Save cropping coordinates for subsequent use

In [ ]:
coords = jvis.get_cropcoords(maxp, cropping_stream)
np.save(os.path.join(directory,'coords.npy'), coords)
coords

## Create trimmed npy files for resampling

In [ ]:
%%time
with h5py.File(hdf5_file, 'r') as f:
    slc = (slice(None,None), slice(coords[0,0],coords[0,1]), slice(coords[1,0],coords[1,1]))
    np.save(os.path.join(directory,'fos_647.npy'), f['fos_647'][slc])
    np.save(os.path.join(directory,'fos_790.npy'), f['fos_790'][slc])
    np.save(os.path.join(directory,'auto.npy'), f['auto'][slc])
ws.info()

## Initialize alignment 

First initialize the atlas reference files by roughly slicing them to the part of the brain under consideration. 


In [ ]:
#init atals and reference files
annotation_directory = '/home/zachpen87/clearmap/AtlasDocs/Horizontal'

#annotation files
annotation_file, reference_file=ano.prepare_annotation_files(
    annotation_file = os.path.join(annotation_directory,'ABA_25um_annotation.tif'),
    reference_file = os.path.join(annotation_directory,'ABA_25um_reference.tif'),
    slicing=(slice(None),
             slice(None),
             slice(None)), 
    orientation=(1,2,3),
    overwrite=False, verbose=True);

#alignment parameter files    
align_channels_affine_file   = io.join(resources_directory, 'Alignment/align_affine.txt')
align_reference_affine_file  = io.join(resources_directory, 'Alignment/align_affine.txt')
align_reference_bspline_file = io.join(resources_directory, 'Alignment/align_bspline.txt')

## Resampling and atlas alignment 

### Resampling 
* `source_resolution` is the resolution of the data as `(x-resolution, y-resolution, z-resolution)`.  
* `sink-resolution` is the resolution of the Atlas. We use the 25µm version of the annotation.   

In [ ]:
%%time
resample_parameter = {
    "source_resolution" : (5,5,4.5),
    "sink_resolution"   : (25,25,25),
    "orientation" : (3,2,1),
    "processes" : 12,
    "verbose" : False}
    
res.resample(
    ws.filename('raw', postfix='647'),
    sink=ws.filename('resampled', postfix='647'),
    **resample_parameter)

res.resample(
    ws.filename('raw', postfix='790'),
    sink=ws.filename('resampled', postfix='790'),
    **resample_parameter)

res.resample(
    ws.filename('autofluorescence'), 
    sink=ws.filename('resampled', postfix='autofluorescence'), 
    **resample_parameter)

for npy in ['auto.npy','fos_647.npy','fos_790.npy']: os.remove(os.path.join(directory,npy))

### Alignment 

This step interfaces to [elastix](https://elastix.lumc.nl/) to aligin the resampled raw image to the resampled autofluorescence image.

As the autofluorescence image is often taken in a separate step with different microscope settings both images typically do not align. This steps corrects for this via a affine transformation.

In [ ]:
%%time

channel_list = ['647', '790']
for channel in channel_list:
    print('Alligning channel {x} with autofluorescence'.format(x=channel))
    align_channels_parameter = {            
        "moving_image" : ws.filename('resampled', postfix='autofluorescence'),
        "fixed_image"  : ws.filename('resampled', postfix = channel),
        "affine_parameter_file"  : align_channels_affine_file,
        "bspline_parameter_file" : None,
        "result_directory" :  ws.filename('resampled_to_auto', postfix = channel)} 
    elx.align(**align_channels_parameter);

print('Alligning reference with autofluorescence')
align_reference_parameter = {            
    "moving_image" : reference_file,
    "fixed_image"  : ws.filename('resampled', postfix='autofluorescence'),
    "affine_parameter_file"  :  align_reference_affine_file,
    "bspline_parameter_file" :  align_reference_bspline_file,
    "result_directory" :  ws.filename('auto_to_reference')}
elx.align(**align_reference_parameter);

## Visualize Allignment

Below, alignment can be visualized by comparing the transformed Allen Brain Atlas reference image to the resampled autofluorescence and signal channels.

In [ ]:
%output size=100

plane = 'h'   # h, s, or c.  for horizontal, sagittal, or coronal
inter = 20    # interval between slices
channels = ['647','790'] #list of channel postfix if applicable, or None

align_imgs = jvis.load_alignment_images(ws, channels=channels)
auto_hmap = jvis.gen_hmap(align_imgs['auto'],plane,title='Autofluoresence',inter=inter)
auto_sig_hmap = jvis.gen_hmap(align_imgs['auto_to_sig'],plane,title='Autofluoresence to Sig',inter=inter)
sig1_hmap = jvis.gen_hmap(align_imgs['647'],plane,title='Signal_647',inter=inter, lims=(50,300))
sig2_hmap = jvis.gen_hmap(align_imgs['790'],plane,title='Signal_790',inter=inter, lims=(50,200))
ref_hmap = jvis.gen_hmap(align_imgs['reference'],plane,title='Transformed ABA Reference',inter=inter) 

pointer = streams.PointerXY(x=0, y=0, source=ref_hmap)
def cross_hair_info(x, y):
    return hv.HLine(y) * hv.VLine(x) 
dmap = hv.DynamicMap(cross_hair_info, streams=[pointer])

hv.output(widget_location='bottom')
((ref_hmap+auto_hmap+auto_sig_hmap+sig2_hmap+sig1_hmap)*dmap).cols(5)    